# Подготовка данных

In [15]:
#импортируем необходимые библиотеки и отключаем предупреждения Anaconda
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [16]:
#смотрим на наши данные
data = pd.read_csv('adult.csv')
data.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education.num     32561 non-null int64
marital.status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital.gain      32561 non-null int64
capital.loss      32561 non-null int64
hours.per.week    32561 non-null int64
native.country    32561 non-null object
income            32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [17]:
#заменяем пропущенные значения NaN значениями и смотрим в каких признаках у нас пропуски
data.replace(to_replace='?', value = np.nan, inplace = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         30725 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education.num     32561 non-null int64
marital.status    32561 non-null object
occupation        30718 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital.gain      32561 non-null int64
capital.loss      32561 non-null int64
hours.per.week    32561 non-null int64
native.country    31978 non-null object
income            32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [18]:
#заменяем NaN значения в этих признаках на моду.
for c in ["workclass", "occupation", "native.country"]:
    data[c].fillna(data[c].mode()[0], inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education.num     32561 non-null int64
marital.status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital.gain      32561 non-null int64
capital.loss      32561 non-null int64
hours.per.week    32561 non-null int64
native.country    32561 non-null object
income            32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [19]:
#отделяем целевой признак
X = data.drop(['income'], axis = 1)
y = data['income']
y.replace(to_replace='>50K', value = 1, inplace = True)
y.replace(to_replace='<=50K', value = 0, inplace = True)

In [20]:
#разбиваем на тестовые и обучающие данные
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 17)

In [21]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

#маска для отбора категориальных признаков
categorical_feature_mask = X_train.dtypes==object
#отбираем только категориальные признаки используя маску и добавляем их в список
categorical_cols = X.columns[categorical_feature_mask].tolist()
#модели не умеют обучатся на текстовых данных поэтому мы преобразуем их в числовое значание
for feature in categorical_cols:
        X_train[feature] = le.fit_transform(X_train[feature])
        X_test[feature] = le.transform(X_test[feature])

In [22]:
#нормализируем признаки
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X.columns)

X_test = pd.DataFrame(scaler.transform(X_test), columns = X.columns)

## Логистическая регрессия

In [23]:
from sklearn.linear_model import LogisticRegression #импортируем логистическую регрессию
from sklearn.metrics import mean_squared_error #будем использовать среднюю квадратическую ошибку для оценки моделей

logreg = LogisticRegression(random_state = 17) #иницилиазируем модель
logreg.fit(X_train, y_train) #обучаем модель
y_pred = logreg.predict(X_test) #делаем предсказание

print("Mean squared error: %.3f" %  mean_squared_error(y_test, y_pred)) #вычисляем среднюю квадратическую ошибку

Mean squared error: 0.175


## Lasso-регрессия

In [24]:
#ищем оптимальный параметр альфа при кросс-валлидации 5
from sklearn.linear_model import LassoCV, Lasso 
lasso_cv = LassoCV(random_state = 17, cv = 5) 
lasso_cv.fit(X_train, y_train) 
lasso_cv.alpha_

0.00014405702595337978

In [26]:
lasso = Lasso(alpha = 0.01, random_state = 17)
lasso.fit(X_train, y_train) 
y_pred = lasso.predict(X_test)
print("Mean squared error: %.3f" %  mean_squared_error(y_test, y_pred))

Mean squared error: 0.134


## Ridge-регрессия


In [27]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=1, random_state = 17)
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_test)
print("Mean squared error: %.3f" %  mean_squared_error(y_test, y_pred))

Mean squared error: 0.134


## RandomForest без параметров

In [28]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(random_state = 17, n_estimators=100, n_jobs=-1)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)
print("Mean squared error: %.3f" %  mean_squared_error(y_test, y_pred))

Mean squared error: 0.143


## Поиск лучших параметров для случайного леса

In [90]:
%%time
from sklearn.model_selection import GridSearchCV, StratifiedKFold

parameters = {'max_features': [1, 2, 4, 6, 8, 10], 
              'min_samples_leaf': [3, 5, 7, 9, 11, 13], 
              'max_depth': [5,10,15]}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)

forest_best = GridSearchCV(forest, parameters, scoring='roc_auc', n_jobs=-1, cv=skf, verbose=1)
forest_best.fit(X_train, y_train)
print(forest_best.best_params_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   55.8s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:  3.8min finished


{'max_depth': 15, 'max_features': 2, 'min_samples_leaf': 3}
CPU times: user 5.21 s, sys: 364 ms, total: 5.57 s
Wall time: 3min 46s


## Случайный лес с лучшими параметрами

In [29]:
forest_with_parameters = RandomForestClassifier(random_state = 17, n_estimators=100, n_jobs=-1, max_depth=15, 
                                max_features=2, min_samples_leaf=3)
forest_with_parameters.fit(X_train, y_train)
y_pred = forest_with_parameters.predict(X_test)
print("Mean squared error: %.3f" %  mean_squared_error(y_test, y_pred))

Mean squared error: 0.137


## Бэггинг на логистических регрессиях

In [30]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import RandomizedSearchCV
parameters = {'max_features': [4,6,8,10], 'max_samples': [0.5, 0.7, 0.9], 
              "base_estimator__C": [0.0001, 0.001, 0.01, 1, 10, 100]}

In [91]:
%%time
#ищем лучшие параметры
bc = BaggingClassifier(base_estimator=logreg, n_estimators=100, n_jobs=-1, random_state=17)
rsb = RandomizedSearchCV(estimator=bc, cv=skf, param_distributions=parameters, scoring='roc_auc', n_jobs=-1, random_state=17)
rsb.fit(X_train, y_train)
print(rsb.best_score_, rsb.best_params_)

0.8535407018151622 {'max_samples': 0.7, 'max_features': 8, 'base_estimator__C': 1}
CPU times: user 537 ms, sys: 166 ms, total: 703 ms
Wall time: 1min 11s


In [31]:
bc = BaggingClassifier(base_estimator=logreg, n_estimators=100, n_jobs=-1, random_state=17, max_samples=0.7, 
                       max_features=8)
bc.fit(X_train, y_train)
y_pred = bc.predict(X_test)
print("Mean squared error: %.3f" %  mean_squared_error(y_test, y_pred))

Mean squared error: 0.178
